In [1]:
from bs4 import BeautifulSoup
from decimal import *
import requests
import requests
import numpy as np
import pandas as pd
import unicodedata as ud
import re
import urllib2
import codecs
import math


In [2]:
def save_string_field(dict, key, value, def_str, preamble):
    if len(value) > 0:
        if preamble == "":
            dict[key] = ud.normalize('NFKD', value).encode('ascii', 'ignore')
        else:
            dict[key] = preamble+value
    else: 
        dict[key] = def_str

def save_numeric_field(dict, key, value):
    if len(value) > 0:
        dict[key] = int(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace(',', ''))
    else: 
        dict[key] = 0

def save_percent_field(dict, key, value):
    if len(value) > 0:
        dict[key] = int(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace('%', ''))
    else: 
        dict[key] = 0

def save_millions_field(dict, key, value):
    nonBreakSpace = u'\xa0'
    value = value.strip(nonBreakSpace)
    if len(value) > 0:
        dict[key] = int(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace(',', '').\
            replace('$','').replace(' million',''))
    else: 
        dict[key] = 0

response = urllib2.urlopen('http://www.ttnews.com/top50/globalfreight/')
html = response.read()
soup = BeautifulSoup(html)
global_freight_columns = ['Rank2015', 'CompanyName', 'Link', 'Headquarters',\
                    'FreightRevenue', 'TotalRevenue', 'FreightRevPercent']
global_freight = pd.DataFrame(data=np.zeros((0,len(global_freight_columns))), columns=global_freight_columns)
global_freight_table = soup.find(id= "rankings_box")
for row in global_freight_table.findAll('tr'):
    col = row.findAll('td')
    data_dict = {}
    j = 0
    for field in col:
        data_dict["Link"] =  ""
        if j == 0:
            if field.get_text().isnumeric():
                save_numeric_field(data_dict, "Rank2015", field.get_text())
            else:
                break
        elif j == 1:
            save_string_field(data_dict, "CompanyName", field.get_text(), "Unknown","")
        elif j == 2:
            save_string_field(data_dict, "Headquarters", field.get_text(),"Unknown","" )
        elif j == 3:
            save_millions_field(data_dict, "FreightRevenue", field.get_text())
        elif j == 4:
            save_millions_field(data_dict, "TotalRevenue", field.get_text())
        else :
            save_percent_field(data_dict, "FreightRevPercent", field.get_text())
        j = j + 1
    single = pd.DataFrame([data_dict], columns=global_freight_columns)
#    print single
    global_freight = global_freight.append(single)
global_freight.to_csv('global_freight.csv')

/Users/satishjoshi/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
